# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import seaborn as sns

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantity_purchased = pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum())
quantity_purchased.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
dist = squareform(pdist(pivot.T, 'euclidean'))
rank = 1/(1+dist)
customer_similarities = pd.DataFrame(rank, index=pivot.columns, columns=pivot.columns)
customer_similarities.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
# most similar for 33:
#customer_similarities
msf33 = customer_similarities[33].sort_values(ascending=False)[1:6]
msf33

CustomerID
3909    0.095358
264     0.093953
3531    0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
most = quantity_purchased.loc[msf33.index]
print(len(most))


for i in msf33.index:
    try:
        print(i,most.loc[i].loc['Salsify, Organic'])
    except:
        pass

282
3531 Quantity    3
Name: Salsify, Organic, dtype: int64
2503 Quantity    1
Name: Salsify, Organic, dtype: int64


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
asdf = most.groupby('ProductName').sum().sort_values(by = 'Quantity', ascending=False)
asdf.head()
#most.groupby('ProductName').agg('Quantity','sum').sort_values('Quantity',ascending=False)

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
index_products = set(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().loc[33].index)
all_products = set(most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).index)
non = set(all_products - index_products)
recommendations = most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).T[non].T.sort_values('Quantity', ascending=False)[0:5]
recommendations

,Quantity
ProductName,
Pecan Raisin - Tarts,3
Bay Leaf,3
Wine - Charddonnay Errazuriz,3
Wine - Ej Gallo Sierra Valley,3
"Oranges - Navel, 72",3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
data = pd.read_csv('../data/customer_product_sales.csv')
quantity_purchased = pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum())
pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)
distances = squareform(pdist(pivot.T, 'euclidean'))
rank=1/(1+distances)
customer_similarities = pd.DataFrame(rank, index=pivot.columns, columns=pivot.columns)

dictionary = {}
for customer_id in pivot.columns:
    msf33 = customer_similarities[customer_id].sort_values(ascending=False)
    most = quantity_purchased.loc[msf33[1:6].index]
    index_products = set(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().loc[customer_id].index)
    all_products = set(most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).index)
    non = set(all_products - index_products)
    recommendations = most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).T[non].T.sort_values('Quantity', ascending=False)[0:5]
    dictionary[customer_id] = recommendations.index

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
final_recomendations = pd.DataFrame(dictionary, index=['First', 'Second', 'Third','Fourth','Fifth']).T
final_recomendations.head()

,First,Second,Third,Fourth,Fifth
33,Pecan Raisin - Tarts,Bay Leaf,Wine - Charddonnay Errazuriz,Wine - Ej Gallo Sierra Valley,"Oranges - Navel, 72"
200,Bay Leaf,Scampi Tail,Pork - Kidney,Muffin - Carrot Individual Wrap,Soup - Campbells Bean Medley
264,Cheese - Parmesan Grated,Soup - Campbells Tomato Ravioli,Muffin - Carrot Individual Wrap,"Pepper - White, Ground",Pork - Kidney
356,Butter - Unsalted,"Pork - Bacon, Double Smoked","Pepper - White, Ground","Veal - Inside, Choice",Sage - Ground
412,Butter - Unsalted,Beef - Top Sirloin,"Pepper - Black, Whole",Table Cloth 81x81 White,Wine - Blue Nun Qualitatswein


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
quantity_purchased = pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum())
pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)
distances = squareform(pdist(pivot.T, 'cosine'))
rank = 1/(1+distances)
customer_similarities = pd.DataFrame(rank, index=pivot.columns, columns=pivot.columns)

dictionary2 = {}
for customer_id in pivot.columns:
    customer_similarities[customer_id].sort_values(ascending=False)
    msf33 = customer_similarities[customer_id].sort_values(ascending=False)
    most = quantity_purchased.loc[msf33[1:6].index]
    index_products = set(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().loc[customer_id].index)
    all_products = set(most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).index)
    non = set(all_products - index_products)
    recommendations = most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).T[non].T.sort_values('Quantity', ascending=False)[0:5]
    dictionary2[customer_id] = recommendations.index
final_recomendations2 = pd.DataFrame(dictionary2, index=['First', 'Second', 'Third','Fourth','Fifth']).T
final_recomendations2.head()

,First,Second,Third,Fourth,Fifth
33,Durian Fruit,Beef - Ground Medium,Bread Crumbs - Japanese Style,Beef - Montreal Smoked Brisket,"Soup - Campbells, Beef Barley"
200,Pasta - Orecchiette,Wine - White Cab Sauv.on,Grouper - Fresh,Apricots - Halves,Pernod
264,"Water - Mineral, Natural",Chicken - Wieners,Cake - Box Window 10x10x2.5,Pickerel - Fillets,Cream Of Tartar
356,Tea - Herbal Sweet Dreams,Ecolab - Lime - A - Way 4/4 L,Chicken - Soup Base,Cream Of Tartar,Rum - Mount Gay Eclipes
412,Wine - Vidal Icewine Magnotta,Crackers - Trio,Wine - Hardys Bankside Shiraz,Gatorade - Xfactor Berry,Mustard - Seed


In [15]:
quantity_purchased = pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum())
pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values ='Quantity').fillna(0)
distances = squareform(pdist(pivot.T, 'cityblock'))
rank = 1/(1+distances)
customer_similarities = pd.DataFrame(rank, index=pivot.columns, columns=pivot.columns)

dictionary3 = {}
for customer_id in pivot.columns:
    customer_similarities[customer_id].sort_values(ascending=False)
    msf33 = customer_similarities[customer_id].sort_values(ascending=False)
    most = quantity_purchased.loc[msf33[1:6].index]
    index_products = set(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().loc[customer_id].index)
    all_products = set(most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).index)
    non = set(all_products - index_products)
    recommendations = most.groupby('ProductName').sum().sort_values('Quantity', ascending=False).T[non].T.sort_values('Quantity', ascending=False)[0:5]
    dictionary3[customer_id] = recommendations.index
final_recomendations3 = pd.DataFrame(dictionary3, index=['First', 'Second', 'Third','Fourth','Fifth']).T
final_recomendations3.head()

,First,Second,Third,Fourth,Fifth
33,Pecan Raisin - Tarts,Bay Leaf,Wine - Charddonnay Errazuriz,Wine - Ej Gallo Sierra Valley,"Oranges - Navel, 72"
200,Bay Leaf,Scampi Tail,Pork - Kidney,Muffin - Carrot Individual Wrap,Soup - Campbells Bean Medley
264,Cheese - Parmesan Grated,Soup - Campbells Tomato Ravioli,Muffin - Carrot Individual Wrap,"Pepper - White, Ground",Pork - Kidney
356,"Veal - Inside, Choice","Pepper - White, Ground",Sage - Ground,Cup - Translucent 7 Oz Clear,Butter - Unsalted
412,Butter - Unsalted,Beef - Top Sirloin,"Pepper - Black, Whole",Table Cloth 81x81 White,Wine - Blue Nun Qualitatswein
